# Advent of code 2022

I'm doing this for fun, so the amount of explanation I put in will vary according to how I feel.

In [1]:
import pandas as pd

## Part 1

Let's have a confusion matrix in a DataFrame:

In [2]:
df=pd.DataFrame({'Rock':{'Rock':3, 'Paper':0, 'Scissors':6},
                'Paper':{'Rock':6, 'Paper':3, 'Scissors':0},
                'Scissors':{'Rock':0, 'Paper':6, 'Scissors':3}
                })

df['Rock']=df['Rock']+1
df['Paper']=df['Paper']+2
df['Scissors']=df['Scissors']+3

df

,Rock,Paper,Scissors
Rock,4,8,3
Paper,1,5,9
Scissors,7,2,6


In [3]:
df=(df
    .rename({'Rock':'A', 'Paper':'B', 'Scissors':'C'}, axis='rows')
    .rename({'Rock':'X', 'Paper':'Y', 'Scissors':'Z'}, axis='columns'))
df

,X,Y,Z
A,4,8,3
B,1,5,9
C,7,2,6


In [4]:
test_input='''A Y
B X
C Z'''

Then just split the input and pump into the matrix to get the final score:

In [5]:
assert sum([df.loc[x, y] for (x, y) in [x.split() for x in test_input.splitlines()]])==15

In [6]:
with open('data/day02.txt') as fIn:
    puzzle_input=fIn.read()

In [7]:
sum([df.loc[x, y] for (x, y) in [x.split() for x in puzzle_input.splitlines()]])

13484

## Part 2

Can also do this as a confusion matrix, but slightly more (mentally) fiddly. It's early, so I'm not going to try to construct the matrix in my head.

In [8]:
df=pd.DataFrame({'Rock':{'Win':'Paper', 'Draw':'Rock', 'Lose':'Scissors'},
                'Paper':{'Win':'Scissors', 'Draw':'Paper', 'Lose':'Rock'},
                'Scissors':{'Win':'Rock', 'Draw':'Scissors', 'Lose':'Paper'}
                })
df

,Rock,Paper,Scissors
Win,Paper,Scissors,Rock
Draw,Rock,Paper,Scissors
Lose,Scissors,Rock,Paper


In [9]:
df=df.replace({'Rock':1, 'Paper':2, 'Scissors':3}).T
df

,Win,Draw,Lose
Rock,2,1,3
Paper,3,2,1
Scissors,1,3,2


In [10]:
df['Win']=df['Win']+6
df['Draw']+=3
df

,Win,Draw,Lose
Rock,8,4,3
Paper,9,5,1
Scissors,7,6,2


In [11]:
df=(df
    .rename({'Rock':'A', 'Paper':'B', 'Scissors':'C'}, axis='rows')
    .rename({'Lose':'X', 'Draw':'Y', 'Win':'Z'}, axis='columns'))
df

,Z,Y,X
A,8,4,3
B,9,5,1
C,7,6,2


In [12]:
test_input='''A Y
B X
C Z'''

Then just run the same statements as last time:

In [13]:
assert sum([df.loc[x, y] for (x, y) in [x.split() for x in test_input.splitlines()]])==12

In [14]:
sum([df.loc[x, y] for (x, y) in [x.split() for x in puzzle_input.splitlines()]])

13433

## Just for fun...

As a little exercise that might be a good way to get the students thinking about reshaping data, let's try starting from a suitable DataFrame:

In [15]:
# Rather ungainly way of creating the dataframe, but
# I'm only really interested in having it for the rest 
# of the exercise.

rps_df=pd.DataFrame(index=pd.Index(['Rock', 'Paper', 'Scissors'], name='P1'),
                    columns=pd.Index(['Rock', 'Paper', 'Scissors'], name='P2'))

for row in rps_df.index:
    for col in rps_df.columns:
        if row==col:
            rps_df.loc[row, col]='Draw'
        elif ((row=='Rock' and col=='Scissors')
              or (row=='Paper' and col=='Rock')
              or (row=='Scissors' and col=='Paper')):
            rps_df.loc[row, col]=rps_df.index.name
rps_df=rps_df.fillna(rps_df.columns.name)
        
rps_df


P2,Rock,Paper,Scissors
P1,,,
Rock,Draw,P2,P1
Paper,P1,Draw,P2
Scissors,P2,P1,Draw


## Part 1

Now, to get into a form so we can easily calculate the score following play:

In [16]:
long_df=(rps_df
         .reset_index()
         .melt(rps_df.index.name, value_name='Outcome'))

long_df

,P1,P2,Outcome
0,Rock,Rock,Draw
1,Paper,Rock,P1
2,Scissors,Rock,P2
3,Rock,Paper,P2
4,Paper,Paper,Draw
5,Scissors,Paper,P1
6,Rock,Scissors,P1
7,Paper,Scissors,P2
8,Scissors,Scissors,Draw


Assume `P1` is the opponent and `P2` is me, then I should be able to calculate the final score fairly easily:

In [17]:
# value of the shape

long_df['P2'].map({'Rock':1, 'Paper':2, 'Scissors':3})

0    1
1    1
2    1
3    2
4    2
5    2
6    3
7    3
8    3
Name: P2, dtype: int64

In [18]:
# Score of the outcome

long_df['Outcome'].map({'P2':6, 'Draw':3, 'P1':0})

0    3
1    0
2    6
3    6
4    3
5    0
6    0
7    6
8    3
Name: Outcome, dtype: int64

In [19]:
long_df['Score']=long_df['P2'].map({'Rock':1, 'Paper':2, 'Scissors':3}) + \
                 long_df['Outcome'].map({'P2':6, 'Draw':3, 'P1':0})
long_df

,P1,P2,Outcome,Score
0,Rock,Rock,Draw,4
1,Paper,Rock,P1,1
2,Scissors,Rock,P2,7
3,Rock,Paper,P2,8
4,Paper,Paper,Draw,5
5,Scissors,Paper,P1,2
6,Rock,Scissors,P1,3
7,Paper,Scissors,P2,9
8,Scissors,Scissors,Draw,6


To make it easy to apply the test input, let's map back to wide form:

In [20]:
long_df.pivot(index='P1', columns='P2', values='Score')


P2,Paper,Rock,Scissors
P1,,,
Paper,5,1,9
Rock,8,4,3
Scissors,2,7,6


And map the `P1` and `P2` values onto the coded values:

In [21]:
pt1_df=(long_df.pivot(index='P1', columns='P2', values='Score')
        .rename({'Rock':'A', 'Paper':'B', 'Scissors':'C'}, axis='rows')
        .rename({'Rock':'X', 'Paper':'Y', 'Scissors':'Z'}, axis='columns'))

pt1_df

P2,Y,X,Z
P1,,,
B,5,1,9
A,8,4,3
C,2,7,6


And apply the test input:

In [22]:
assert sum([pt1_df.loc[x, y] for (x, y) in [x.split() for x in test_input.splitlines()]])==15

And the puzzle input:

In [23]:
sum([pt1_df.loc[x, y] for (x, y) in [x.split() for x in puzzle_input.splitlines()]])

13484

## Part 2

For part 2, we just need to use the desired output as the second value, rather than the player's choice. We can do this by pivoting on a different column:

In [24]:
long_df.pivot(index='P1', columns='Outcome', values='Score')

Outcome,Draw,P1,P2
P1,,,
Paper,5,1,9
Rock,4,3,8
Scissors,6,2,7


And map the `P1` and `P2` values onto the coded values:

In [25]:
pt2_df=(long_df.pivot(index='P1', columns='Outcome', values='Score')
        .rename({'Rock':'A', 'Paper':'B', 'Scissors':'C'}, axis='rows')
        .rename({'P1':'X', 'Draw':'Y', 'P2':'Z'}, axis='columns'))

pt2_df

Outcome,Y,X,Z
P1,,,
B,5,1,9
A,4,3,8
C,6,2,7


In [26]:
assert sum([pt2_df.loc[x, y] for (x, y) in [x.split() for x in test_input.splitlines()]])==12

In [27]:
sum([pt2_df.loc[x, y] for (x, y) in [x.split() for x in puzzle_input.splitlines()]])

13433